In [ ]:
import kagglehub
import pandas
import numpy
import jax
import jax.random as jrnd
import jax.numpy as jnp
import flax.linen as nn
from flax.training import train_state
import optax

path = kagglehub.dataset_download("oddrationale/mnist-in-csv")

path_train = path + "/mnist_train.csv"
path_test = path + "/mnist_test.csv"

df_train = pandas.read_csv(path_train)
df_test = pandas.read_csv(path_test)

TRAIN_SIZE = 60000
TEST_SIZE = 10000
IMG_SIZE = (28, 28)

In [ ]:
X_train = jnp.array(df_train.iloc[:, 1:].to_numpy().reshape(TRAIN_SIZE, IMG_SIZE[0], IMG_SIZE[1]))
Y_train = jnp.array(df_train.iloc[:, 0].to_numpy().reshape(TRAIN_SIZE, 1))

X_test = jnp.array(df_test.iloc[:, 1:].to_numpy().reshape(TEST_SIZE, IMG_SIZE[0], IMG_SIZE[1]))
Y_test = jnp.array(df_test.iloc[:, 0].to_numpy().reshape(TEST_SIZE, 1))

In [ ]:
class Model():
    @nn.compact
    def __call__(self, x):
        x = nn.leaky_relu(nn.Conv(features=8, kernel_size=(5,5)))
        x = nn.leaky_relu(nn.Conv(features=12, kernel_size=(3,3)))
        x = nn.max_pool(x, window_shape=(2,2), strides=(2,2))
        x = nn.leaky_relu(nn.Conv(features=6, kernel_size=(3,3)))
        x = nn.max_pool(x, window_shape=(2,2), strides=(2,2))# (6,7,7) hopefully
        x = x.reshape((x.shape[0], -1))
        x = nn.leaky_relu(nn.Dense(features=80)(x))
        x = nn.leaky_relu(nn.Dense(features=320)(x))
        x = nn.leaky_relu(nn.Dense(features=120)(x))
        x = nn.leaky_relu(nn.Dense(features=30)(x))
        x = nn.leaky_relu(nn.Dense(features=10)(x))
        return x

def create_model_state(rng, model, lr):
    params = model.init(rng, jnp.zeros(X_train[0]))
    optim = optax.adam(lr)
    return train_state.TrainState()